## One DOF ussing Deep XDE

In [1]:
#conda install -c conda-forge deepxde

In [10]:
import numpy as np
import tensorflow as tf
def pde(x,y) :
    dy_x = tf.gradients(y,x)[0]
    dy_x, dy_t = dy_x[:, 0:1], dy_x[:, 1:2]
    dy_xx = tf.gradients(dy_x,x)[0][:, 0:1]
    return dy_t + y*dy_x - 0.01 / np.pi * dy_xx

In [11]:
import deepxde as dde
# Geometry and time domain
geom = dde.geometry.Interval(-5,5)
timedomain = dde.geometry.TimeDomain(0, 9.99)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [12]:
#bc = 0
bc = dde.DirichletBC(geomtime, lambda x:0, lambda _, on_boundary: on_boundary)

In [13]:
#ic = -sin(pi*x)
ic = dde.IC(geomtime, lambda x: -np.sin(np.pi*x[:, 0:1]), lambda _, on_initial: on_initial)

In [14]:
#Generate data
data = dde.data.TimePDE(geomtime, pde, [bc, ic], num_domain = 30000, num_boundary = 1000, num_initial = 200)

In [15]:
# The NN
net = dde.maps.FNN([2]+[20]+[20] + [1],'tanh','Glorot normal')
model = dde.Model(data,net)
model.compile('adam', lr = 1e-3)

Compiling model...
Building feed-forward neural network...
'build' took 0.045072 s



2023-06-07 10:38:35.748512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 10:38:35.748723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 10:38:35.748884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

'compile' took 0.288249 s



In [16]:
losshistory, train_state = model.train(epochs = 30000)

Training model...



2023-06-07 10:39:11.489399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-07 10:39:11.591306: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f23c4003740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-07 10:39:11.591337: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 with Max-Q Design, Compute Capability 6.1
2023-06-07 10:39:11.650350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-07 10:39:13.172552: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If build

InternalError: Graph execution error:

2 root error(s) found.
  (0) INTERNAL: RET_CHECK failure (tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:618) dnn != nullptr 
	 [[{{node cluster_0_1/xla_compile}}]]
	 [[cluster_0_1/merge_oidx_0/_1]]
  (1) INTERNAL: RET_CHECK failure (tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:618) dnn != nullptr 
	 [[{{node cluster_0_1/xla_compile}}]]
0 successful operations.
0 derived errors ignored.